<a href="https://colab.research.google.com/github/GuilhermeSotti/Aula-de-SO/blob/master/Tensorflow_Code5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
import matplotlib.pyplot as plt

def main():

  local_weights_file = './inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
  base_dir = './train_images/'
  pre_trained_model, last_output = load_model_previously_created(local_weights_file)
  model = recreate_net(pre_trained_model, last_output)
  train_gen, validation_gen = format_input_data(base_dir)
  history = train_net(model, train_gen, validation_gen)
  acc, val_acc, loss, val_loss, epochs = calc_performance(history)
  show_graphic_performance(acc, val_acc, loss, val_loss, epochs)

def load_model_previously_created(local_weights_file):

  pre_trained_model = InceptionV3(
                        input_shape = (150, 150, 3),
                        include_top = False,
                        weights = None
  )
  pre_trained_model.load_weights(local_weights_file)

  for layers in pre_trained_model.layers:
    layers.trainable = False

  pre_trained_model.sumary()
  last_layers = pre_trained_model.get_layers('mixed7')
  print('last layers output shape: ', last_layers.output_shape)
  last_output = last_layers.output

  return pre_trained_model, last_output

def recreate_net(pre_trained_model, last_output):

  x = layers.Flatten()(last_output)
  x = layers.Dense(1024, activation  = 'relu')(x)
  x = layers.Dropout(0.2)(x)
  x = layers.Dense(1, activation = 'sigmoid')(x)

  RMSprop = RMSprop(lr = 0.0001,
                rho = 0.9,
                momentum = 0.0,
                epsilon = 1e-07,
                centered = True,
                name = "Test",
                **kwargs
  )
  
  model = Model(pre_trained_model.input, x)
  model.compile(optimizer = RMSprop,
                loss = 'binary_crossentropy',
                metrics = ['accuracy']
  )

  return model

def format_input_data(base_dir):

  train_dir = os.path.join(base_dir, 'train')
  train_cats_dir = os.path.join(train_dir, 'cats')
  train_dogs_dir = os.path.join(train_dir, 'dogs')
  train_cats_fNames = os.listdir(train_cats_dir)
  train_dogs_fNames = os.listdir(train_dogs_dir)

  validation_dir = os.path.join(base_dir, 'validation')
  validation_cats_dir = os.path.join(validation_dir, 'cats')
  validation_dogs_dir = os.path.join(validation_dir, 'dogs')

  train_data_gen = ImageDataGenerator(
                      rescale = 1. / 255.,
                      rotation_range = 40,
                      width_shift_range = 0.2,
                      heigth_shift_range = 0.2,
                      shear_range = 0.2,
                      zoom_range = 0.2,
                      horizontal_flip = True
  )
  test_data_gen = ImageDataGenerator(rescale = 1. / 255)

  train_gen = train_data_gen.flow_from_directory(
                train_dir,
                batch_size = 20,
                class_mode = 'binary',
                target_size = (150, 150)
  )
  
  validation_gen = test_data_gen.flow_from_directory(
                      validation_dir,
                      batch_size = 20,
                      class_mode = 'binary',
                      target_size = (150, 150)
  )

  return train_gen, validation_gen

def train_net(model, train_gen, validation_gen):

  history = model.fit(
              train_gen,
              validation_data = validation_gen,
              steps_per_epochs = 100,
              epochs = 20,
              validation_steps = 50,
              verbose = 2
  )

  return history

def calc_performance(history):

  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(len(acc))

  return acc, val_acc, loss, val_loss, epochs

def show_graphic_performance(acc, val_acc, loss, val_loss, epochs):

  plt.plot(epochs, acc, 'r', label = 'Trainig Accuracy')
  plt.plot(epochs, val_acc, 'b', label = 'Validation Accuracy')
  plt.title('Training and Validation Accuracy')
  plt.legend(loc = 0)
  plt.figure()
  plt.plot(epochs, loss, 'r', label = 'Training Loss')
  plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
  plt.title('Training and Validation Loss')
  plt.legend(loc = 0)
  plt.figure()
  plt.show()

if __name__ == '__main__':
  main()
